In [1]:
import os
os.system("apt -q -qq install -y /kaggle/input/rsna_2024_o3d_color_vox/other/default/7/liblzma-dev_5.2.5-2ubuntu1_amd64.deb")
os.system("apt -q -qq install -y /kaggle/input/rsna_2024_o3d_color_vox/other/default/7/libunwind-dev_1.3.2-2build2.1_amd64.deb")

os.system("cp /usr/lib/x86_64-linux-gnu/libunwind.so /usr/lib/x86_64-linux-gnu/libunwind.so.1")



dpkg-preconfigure: unable to re-open stdin: No such file or directory


Selecting previously unselected package liblzma-dev:amd64.
(Reading database ... 123110 files and directories currently installed.)
Preparing to unpack .../liblzma-dev_5.2.5-2ubuntu1_amd64.deb ...
Unpacking liblzma-dev:amd64 (5.2.5-2ubuntu1) ...
Setting up liblzma-dev:amd64 (5.2.5-2ubuntu1) ...




dpkg-preconfigure: unable to re-open stdin: No such file or directory


Selecting previously unselected package libunwind-dev:amd64.
(Reading database ... 123150 files and directories currently installed.)
Preparing to unpack .../libunwind-dev_1.3.2-2build2.1_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2.1) ...
Setting up libunwind-dev:amd64 (1.3.2-2build2.1) ...
Processing triggers for man-db (2.10.2-1) ...


0

In [2]:
%pip install --quiet /kaggle/input/timm_3d_deps/other/initial/10/pydicom/pydicom/pydicom-2.4.4-py3-none-any.whl
%pip install timm_3d --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/timm_3d/
%pip install torchio --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/torchio/
%pip install itk --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/itk/itk
%pip install skorch --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/skorch/skorch
%pip install spacecutter --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/spacecutter/
%pip install pgzip --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/pgzip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install open3d --no-index --quiet --find-links=/kaggle/input/timm_3d_deps/other/initial/10/open3d/
%pip uninstall -y --quiet open3d
%pip install --quiet /kaggle/input/rsna_2024_o3d_color_vox/other/default/7/open3d_cpu-0.18.0-cp310-cp310-linux_x86_64.whl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Stage 1: Predict keypoints

In [4]:
import sys
import os
from pathlib import Path
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.patches import Polygon

sys.path.insert(0, '/kaggle/input/spinenet/SpineNet')
import spinenet
from spinenet import SpineNet, download_example_scan
from spinenet.io import load_dicoms_from_folder

spnt = SpineNet(device='cuda:0', verbose=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading Detection Model...


/kaggle/input/spinenet/SpineNet/spinenet/models/vfr.py:186: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(latest_pt, map_location="cpu")


==> Loading model trained for 436 epochs...
Loading Appearance Model...
==> Loading model trained for 188 epochs...
Loading Context Model...
==> Loading model trained for 17 epochs...


/kaggle/input/spinenet/SpineNet/spinenet/models/appearance.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(latest_pt, map_location="cpu")
/kaggl

Loading Grading Model...


/kaggle/input/spinenet/SpineNet/spinenet/models/grading.py:333: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(latest_pt, map_location="cpu")


==> Loading model trained for 2 epochs...


In [5]:
LEVELS = ["L1", "L2", "L3", "L4", "L5", "S1"]
COLORS = {
    "L1": "red",
    "L2": "blue",
    "L3": "green",
    "L4": "yellow",
    "L5": "white",
    "S1": "purple"
}

In [6]:
test_descs_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv"
test_images_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images"

In [7]:
test_descs = pd.read_csv(test_descs_path)
test_descs_filtered = test_descs[test_descs["series_description"] == "Sagittal T1"]
test_descs

,study_id,series_id,series_description
0,44036939,2828203845,Sagittal T1
1,44036939,3481971518,Axial T2
2,44036939,3844393089,Sagittal T2/STIR


In [8]:
def calculate_centers(data):
    centers = {}
    for item in data:
        level = item["predicted_label"]
        if level in LEVELS:
            average_polygon = item["average_polygon"]
            centroid_x = np.mean(average_polygon[:, 0])
            centroid_y = np.mean(average_polygon[:, 1])
            centroid_z = item["slice_nos"][len(item["slice_nos"])//2]
            centers[level] = (centroid_x, centroid_y, centroid_z)
    return centers

In [9]:
centers_per_study = {
    "study_id": [],
    "series_id": [],
    "x": [],
    "y": [],
    "instance_number": [],
    "level": []
}

In [10]:
for index, row in test_descs_filtered.iterrows():
    scan = load_dicoms_from_folder(f"{test_images_path}/{row['study_id']}/{row['series_id']}", require_extensions=False)
    num_slices = scan.volume.shape[-1]

    vert_dicts = spnt.detect_vb(scan.volume, scan.pixel_spacing)
    centers = calculate_centers(vert_dicts)
    
    for level in centers:
        centers_per_study["study_id"].append(row['study_id'])
        centers_per_study["series_id"].append(row['series_id'])
        centers_per_study["level"].append(level)
        
        centers_per_study["x"].append(centers[level][0])
        centers_per_study["y"].append(centers[level][1])
        centers_per_study["instance_number"].append(centers[level][2])

In [11]:
centers_per_study = pd.DataFrame.from_dict(centers_per_study)
centers_per_study

,study_id,series_id,x,y,instance_number,level
0,44036939,2828203845,335.785366,154.553153,10,L1
1,44036939,2828203845,304.148934,226.185392,11,L2
2,44036939,2828203845,272.576395,293.824498,13,L3
3,44036939,2828203845,248.552402,365.867719,14,L4
4,44036939,2828203845,253.116013,451.840578,14,L5
5,44036939,2828203845,302.432602,525.408792,14,S1


# Stage 1.5: Dump out bounding boxes csv

In [12]:
def convert_coords_to_patient(x, y, dicom_slice):            
    dX, dY = dicom_slice.PixelSpacing
    
    X = np.array(list(dicom_slice.ImageOrientationPatient[:3]) + [0]) * dY
    Y = np.array(list(dicom_slice.ImageOrientationPatient[3:]) + [0]) * dX

    S = np.array(list(dicom_slice.ImagePositionPatient) + [1])

    transform_matrix = np.array([Y, X, np.zeros(len(X)), S]).T
    # transform_matrix = transform_matrix @ transform_matrix_factor

    return (transform_matrix @ np.array([y, x, 0, 1]).T)

In [13]:
from pydicom import dcmread

test_images_basepath = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images"

patient_coords_dict = {
    "study_id": [],
    "level": [],
    "x": [],
    "y": [],
    "z": []
}

for index, group in centers_per_study.groupby("study_id"):
    for row_index, row in group.iterrows():
        dicom_slice_path = f"{test_images_basepath}/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm"
        dicom_slice = dcmread(dicom_slice_path)
        coords = convert_coords_to_patient(row['x'], row['y'], dicom_slice)
        
        patient_coords_dict["study_id"].append(row['study_id'])
        patient_coords_dict["level"].append(row['level'])
        patient_coords_dict["x"].append(coords[0])
        patient_coords_dict["y"].append(coords[1])
        patient_coords_dict["z"].append(coords[2])
    
patient_coords = pd.DataFrame.from_dict(patient_coords_dict)
patient_coords

,study_id,level,x,y,z
0,44036939,L1,20.724218,44.669312,99.955032
1,44036939,L2,16.510600,32.805650,73.256503
2,44036939,L3,9.367907,20.965948,48.189862
3,44036939,L4,5.147208,11.956951,21.337373
4,44036939,L5,3.686905,13.668305,-10.869371
5,44036939,S1,2.437303,32.162026,-38.429146


In [14]:
patient_bounding_boxes_dict = {
    "study_id": [],
    "level": [],
    "x_min": [],
    "y_min": [],
    "z_min": [],
    "x_max": [],
    "y_max": [],
    "z_max": [],
}

for index, group in patient_coords.groupby("study_id"):
    ordered_group = group.sort_values(by="level", ascending=True)
    if len(ordered_group) != 6:
        continue
    for level_index in range(5):
        patient_bounding_boxes_dict["study_id"].append(ordered_group['study_id'].iloc[0])
        level_label = ordered_group['level'].iloc[level_index].lower() + "_" + ordered_group['level'].iloc[level_index + 1] 
        patient_bounding_boxes_dict["level"].append(level_label)
        
        # Middle vertebra points
        pt_0 = np.array(ordered_group.iloc[level_index][["x", "y", "z"]])
        pt_1 = np.array(ordered_group.iloc[level_index + 1][["x", "y", "z"]])
        
        # Distance vector to the next vertebra
        d_vec = np.array(pt_0 - pt_1)
        d_size = np.linalg.norm(d_vec)
        d_unit = d_vec / d_size
        
        
        # Get a pair of orthogonal vectors to find x and y boundary candidates
        orth_1 = np.random.randn(3).astype(np.float64)
        orth_1 = orth_1 - orth_1.dot(d_unit) * d_unit
        orth_1 = orth_1 / np.linalg.norm(orth_1)
        
        orth_1 = orth_1.astype(np.float64)
        d_unit = d_unit.astype(np.float64)
        
        orth_2 = np.cross(orth_1, d_unit)
        orth_2 = orth_2.astype(np.float64)
        
        orth_1 *= d_size
        orth_2 *= d_size
        
        # Get candidate points (10 of them, 2 per orthogonal per each vertebra center, and the centers themselves)
        c_pts = np.array([pt - vec for pt in (pt_0, pt_1) for vec in (orth_1, orth_2)] + 
                         [pt + vec for pt in (pt_0, pt_1) for vec in (orth_1, orth_2)] +
                         [pt_0, pt_1])
        
        # x_min and x_max are just the min and max from all this
        x_min = np.min(c_pts[:, 0])
        x_max = np.max(c_pts[:, 0])
        
        x_delta = x_max - x_min
        x_min -= x_delta * 0.25
        x_max += x_delta * 0.25
        
        # y_max is going to be over the center ys
        # And we're going to get y_min by getting y_min over c_pts and then extending the y_min over center ys
        c_pts_y_min = np.min(c_pts[:, 1])
        c_pts_y_max = np.max(c_pts[:, 1])

        y_max = max(pt_0[1], pt_1[1])
        y_min = min(pt_0[1], pt_1[1])
        
        y_max += abs(c_pts_y_max - y_max) * 2
        y_min -= abs(c_pts_y_min - y_min) / 2
        
        # z_max and z_min will be the same as x_min and x_max
        z_min = np.min(c_pts[:, 2])
        z_max = np.max(c_pts[:, 2])    
        
        patient_bounding_boxes_dict["x_min"].append(x_min)
        patient_bounding_boxes_dict["y_min"].append(y_min)
        patient_bounding_boxes_dict["z_min"].append(z_min)
        patient_bounding_boxes_dict["x_max"].append(x_max)
        patient_bounding_boxes_dict["y_max"].append(y_max)
        patient_bounding_boxes_dict["z_max"].append(z_max)

patient_bounding_boxes = pd.DataFrame.from_dict(patient_bounding_boxes_dict)
patient_bounding_boxes

,study_id,level,x_min,y_min,z_min,x_max,y_max,z_max
0,44036939,l1_L2,-20.229776,21.325889,63.163957,57.464594,90.588355,110.047577
1,44036939,l2_L3,-29.742724,8.676290,37.003153,55.621231,81.964284,84.443213
2,44036939,l3_L4,-31.575526,0.925460,11.516468,46.090641,65.091910,58.010767
3,44036939,l4_L5,-38.062350,-1.812389,-13.090067,46.896464,68.745666,23.558069
4,44036939,l5_S1,-46.412285,0.146727,-56.241432,52.536494,86.248336,6.942915


# Stage 2: Run inference on individual vertebrae

## Data Loading

In [15]:
import os

def retrieve_image_paths(base_path, study_id, series_id):
    series_dir = os.path.join(base_path, str(study_id), str(series_id))
    images = os.listdir(series_dir)
    image_paths = [os.path.join(series_dir, img) for img in images]
    return image_paths

In [16]:
import open3d as o3d
import pgzip
import os

def read_vertebral_levels_as_voxel_grids(dir_path,
                                         vertebral_levels: list[str],
                                         max_bounds: list[np.array],
                                         min_bounds: list[np.array],
                                         pcd_overall: o3d.geometry.PointCloud = None,
                                         cache_basepath="/kaggle/working/cached_3d",
                                         series_type_dict=None,
                                         downsampling_factor=1,
                                         voxel_size=(128, 128, 42),
                                        caching=False):
    ret = {}
    
    resize = tio.Resize(voxel_size)

    if pcd_overall is None:
        pcd_overall = read_study_as_pcd(dir_path,
                                        series_types_dict=series_type_dict,
                                        downsampling_factor=downsampling_factor,
                                        img_size=(voxel_size[0], voxel_size[2]),
                                        stack_slices_thickness=True,
                                        resize_slices=False)

    
    for index, vertebral_level in enumerate(vertebral_levels):
            bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bounds[index], max_bound=max_bounds[index])
            pcd_level = pcd_overall.crop(bbox)

            size = 1
            voxel_level = o3d.geometry.VoxelGrid().create_from_point_cloud(pcd_level, size,
                                                                           color_mode=o3d.geometry.VoxelGrid.VoxelColorMode.MAX)

            coords = np.array([voxel.grid_index for voxel in voxel_level.get_voxels()])
            vals = np.array([voxel.color for voxel in voxel_level.get_voxels()], dtype=np.float16)

            size = np.max(coords, axis=0) + 1
            grid = np.zeros((3, size[0], size[1], size[2]), dtype=np.float32)
            indices = coords[:, 0], coords[:, 1], coords[:, 2]

            for i in range(3):
                grid[i][indices] = vals[:, i]

            grid = resize(grid)

            ret[vertebral_level] = grid

    return ret


In [17]:
def read_study_as_pcd(dir_path,
                      series_types_dict=None,
                      downsampling_factor=1,
                      resize_slices=True,
                      resize_method="nearest",
                      stack_slices_thickness=True,
                      img_size=(256, 256)):
    pcd_overall = o3d.geometry.PointCloud()

    for path in glob.glob(os.path.join(dir_path, "**/*.dcm"), recursive=True):
        dicom_slice = dcmread(path)

        series_id = os.path.basename(os.path.dirname(path))
        study_id = os.path.basename(os.path.dirname(os.path.dirname(path)))
        if series_types_dict is None or int(series_id) not in series_types_dict:
            series_desc = dicom_slice.SeriesDescription
        else:
            series_desc = series_types_dict[int(series_id)]
            series_desc = series_desc.split(" ")[-1]

        x_orig, y_orig = dicom_slice.pixel_array.shape
        if resize_slices:
            if resize_method == "nearest":
                img = np.expand_dims(cv2.resize(dicom_slice.pixel_array, img_size, interpolation=cv2.INTER_AREA), -1)
            elif resize_method == "maxpool":
                img_tensor = torch.tensor(dicom_slice.pixel_array).float()
                img = F.adaptive_max_pool2d(img_tensor.unsqueeze(0), img_size).numpy()
            else:
                raise ValueError(f"Invalid resize_method {resize_method}")
        else:
            img = np.expand_dims(np.array(dicom_slice.pixel_array), -1)
        x, y, z = np.where(img)

        downsampling_factor_iter = max(downsampling_factor, int(math.ceil(len(x) / 6e6)))

        index_voxel = np.vstack((x, y, z))[:, ::downsampling_factor_iter]
        grid_index_array = index_voxel.T
        pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(grid_index_array.astype(np.float64)))

        vals = np.expand_dims(img[x, y, z][::downsampling_factor_iter], -1)
        if series_desc == "T1":
            vals = np.pad(vals, ((0, 0), (0, 2)))
        elif series_desc == "T2":
            vals = np.pad(vals, ((0, 0), (1, 1)))
        elif series_desc == "T2/STIR":
            vals = np.pad(vals, ((0, 0), (2, 0)))
        else:
            raise ValueError(f"Unknown series desc: {series_desc}")

        pcd.colors = o3d.utility.Vector3dVector(vals.astype(np.float64))

        if resize_slices:
            transform_matrix_factor = np.matrix(
                [[0, y_orig / img_size[1], 0, 0],
                 [x_orig / img_size[0], 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]]
            )
        else:
            transform_matrix_factor = np.matrix(
                [[0, 1, 0, 0],
                 [1, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]]
            )

        dX, dY = dicom_slice.PixelSpacing
        dZ = dicom_slice.SliceThickness

        X = np.array(list(dicom_slice.ImageOrientationPatient[:3]) + [0]) * dX
        Y = np.array(list(dicom_slice.ImageOrientationPatient[3:]) + [0]) * dY

        S = np.array(list(dicom_slice.ImagePositionPatient) + [1])

        transform_matrix = np.array([X, Y, np.zeros(len(X)), S]).T
        transform_matrix = transform_matrix @ transform_matrix_factor

        if stack_slices_thickness:
            for z in range(int(dZ)):
                pos = list(dicom_slice.ImagePositionPatient)
                if series_desc == "T2":
                    pos[-1] += z
                else:
                    pos[0] += z
                S = np.array(pos + [1])

                transform_matrix = np.array([X, Y, np.zeros(len(X)), S]).T
                transform_matrix = transform_matrix @ transform_matrix_factor

                pcd_overall += copy.deepcopy(pcd).transform(transform_matrix)

        else:
            pcd_overall += copy.deepcopy(pcd).transform(transform_matrix)

    return pcd_overall


In [18]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchio as tio
import torch.nn as nn
import pydicom

CONDITIONS = {
    "Sagittal T2/STIR": ["Spinal Canal Stenosis"],
    "Axial T2": ["Left Subarticular Stenosis", "Right Subarticular Stenosis"],
    "Sagittal T1": ["Left Neural Foraminal Narrowing", "Right Neural Foraminal Narrowing"],
}

LEVELS = ["l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"]

class StudyPerVertebraLevelDataset(Dataset):
    def __init__(self,
                 base_path: str,
                 dataframe: pd.DataFrame,
                 bounds_dataframe: pd.DataFrame,
                 transform_3d=None,
                 vol_size=(128, 128, 128)
                ):
        self.base_path = base_path

        self.dataframe = (dataframe[['study_id', "series_id", "series_description"]]
                          .drop_duplicates())
        self.bounds_dataframe = bounds_dataframe

        self.subjects = self.dataframe[['study_id']].drop_duplicates().reset_index(drop=True)
        self.series = self.dataframe[["study_id", "series_id"]].drop_duplicates().groupby("study_id")[
            "series_id"].apply(list).to_dict()
        self.series_descs = {e[0]: e[1] for e in
                             self.dataframe[["series_id", "series_description"]].drop_duplicates().values}

        self.transform_3d = transform_3d
        self.vol_size = vol_size

    def __len__(self):
        return len(self.subjects)

    def __getitem__(self, index):
        curr = self.subjects.iloc[index % len(self.subjects)]

        study_path = os.path.join(self.base_path, str(curr["study_id"]))
        
        curr_bounds = self.bounds_dataframe[self.bounds_dataframe["study_id"] == curr["study_id"]].sort_values(by="level")
        
        study_images = read_vertebral_levels_as_voxel_grids(study_path,
                                                      vertebral_levels=LEVELS,
                                                      min_bounds=np.array(curr_bounds[['x_min', 'y_min', 'z_min']].values),
                                                      max_bounds=np.array(curr_bounds[['x_max', 'y_max', 'z_max']].values),
                                                      series_type_dict=self.series_descs,
                                                      voxel_size=self.vol_size
                                                    )
        
        ret = []
        for level in LEVELS:
            image = study_images[level]
            image = torch.FloatTensor(image)
            image = self.transform_3d(image)
            ret.append(image.to(torch.half))
            
        return torch.stack(ret), curr["study_id"]

In [19]:
transform_3d = tio.Compose([
    tio.RescaleIntensity([0, 1]),
])

In [20]:
def create_subject_level_testset_and_loader(df: pd.DataFrame,
                                             transform_3d,
                                             base_path: str,
                                             batch_size=1,
                                             num_workers=4):
    testset = StudyPerVertebraLevelDataset(base_path=test_images_basepath, dataframe=df, bounds_dataframe=patient_bounding_boxes, transform_3d=transform_3d)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return testset, test_loader

In [21]:
test_descs_valid = test_descs[test_descs["study_id"].isin(patient_bounding_boxes["study_id"])]

testset, test_loader = create_subject_level_testset_and_loader(test_descs_valid, transform_3d, test_images_basepath)

## Model Loading

In [22]:
import timm_3d
import torch

from spacecutter.losses import CumulativeLinkLoss
from spacecutter.models import LogisticCumulativeLink
from spacecutter.callbacks import AscensionCallback

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

class CustomMaxxVit3dClassifier(nn.Module):
    def __init__(self,
                 backbone,
                 in_chans=3,
                 out_classes=5,
                 cutpoint_margin=0):
        super(CustomMaxxVit3dClassifier, self).__init__()
        self.out_classes = out_classes

        self.config = timm_3d.models.maxxvit.model_cfgs[backbone]

        self.backbone = timm_3d.models.MaxxVit(
            img_size=(128,128,128),
            in_chans=in_chans,
            num_classes=out_classes,
            drop_rate=0,
            drop_path_rate=0,
            cfg=self.config
        )
        self.backbone.head.drop = nn.Dropout(0)
        head_in_dim = self.backbone.head.fc.in_features
        self.backbone.head.fc = nn.Identity()

        self.heads = nn.ModuleList(
            [nn.Sequential(
                nn.Linear(head_in_dim, 1),
                LogisticCumulativeLink(3)
            ) for i in range(out_classes)]
        )

        self.ascension_callback = AscensionCallback(margin=cutpoint_margin)

    def forward(self, x):
        feat = self.backbone(x)
        return torch.swapaxes(torch.stack([head(feat) for head in self.heads]), 0, 1)

    def _ascension_callback(self):
        for head in self.heads:
            self.ascension_callback.clip(head[-1])

model = CustomMaxxVit3dClassifier(backbone="coatnet_rmlp_3_rw", in_chans=3, out_classes=5).to(device)
model.load_state_dict(torch.load("/kaggle/input/rsna_2024_stage_2/pytorch/coatnet_rmlp_3_224_128/9/coatnet_rmlp_3_rw_128_vertebrae_tuned_fold_0_2.pt"))

/opt/conda/lib/python3.10/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/tmp/ipykernel_23/2930394370.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.

<All keys matched successfully>

In [23]:
CONDITIONS = {
    "Sagittal T2/STIR": ["spinal_canal_stenosis"],
    "Axial T2": ["left_subarticular_stenosis", "right_subarticular_stenosis"],
    "Sagittal T1": ["left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"],
}

ALL_CONDITIONS = sorted(["spinal_canal_stenosis", "left_subarticular_stenosis", "right_subarticular_stenosis", "left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"])
LEVELS = ["l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"]

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})

ALL_CONDITIONS


['left_neural_foraminal_narrowing',
 'left_subarticular_stenosis',
 'right_neural_foraminal_narrowing',
 'right_subarticular_stenosis',
 'spinal_canal_stenosis']

In [24]:
# Pre-populate results df
import glob
import os

study_ids = glob.glob("/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/*")
study_ids = [os.path.basename(e) for e in study_ids]

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})
for study_id in study_ids:
    for condition in ALL_CONDITIONS:
        for level in LEVELS:
            row_id = f"{study_id}_{condition}_{level}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": 1/3, "moderate": 1/3, "severe": 1/3}, ignore_index=True)


In [25]:
import torch
from torch.cuda.amp import autocast
import time
import copy

start_time = time.time()

visualize_mid_slices = False

with torch.no_grad():
    with autocast(dtype=torch.float16):
        model.eval()

        for images, study_id in test_loader:
            output = model(images.squeeze(0).to(device))
            for level_index, image in enumerate(images.squeeze(0)):
                if visualize_mid_slices:
                    plt.imshow(np.max(image.numpy()[1, :, :, 62:66], axis=2), cmap="gray")
                    plt.show()
                for condition_index, condition_out in enumerate(output[level_index]):
                    row_id = f"{study_id[0]}_{ALL_CONDITIONS[condition_index]}_{LEVELS[level_index]}"
                    
                    results_df.loc[results_df.row_id == row_id, 'normal_mild'] = condition_out.cpu().numpy()[0]
                    results_df.loc[results_df.row_id == row_id, 'moderate'] = condition_out.cpu().numpy()[1]
                    results_df.loc[results_df.row_id == row_id, 'severe'] = condition_out.cpu().numpy()[2]
                
print("--- %s seconds ---" % (time.time() - start_time))

/tmp/ipykernel_23/2429732810.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):


--- 66.4759669303894 seconds ---


# Stage 2.5: Run inference on the fallback set

## Data Loading

In [26]:
def read_study_as_voxel_grid(dir_path, series_type_dict=None, downsampling_factor=1, img_size=(256, 256)):
    pcd_overall = read_study_as_pcd(dir_path,
                                    series_types_dict=series_type_dict,
                                    downsampling_factor=downsampling_factor,
                                    img_size=img_size)
    box = pcd_overall.get_axis_aligned_bounding_box()

    max_b = np.array(box.get_max_bound())
    min_b = np.array(box.get_min_bound())

    pts = (np.array(pcd_overall.points) - (min_b)) * (
                (img_size[0] - 1, img_size[0] - 1, img_size[0] - 1) / (max_b - min_b))
    coords = np.round(pts).astype(np.int32)
    vals = np.array(pcd_overall.colors, dtype=np.float16)

    grid = np.zeros((3, img_size[0], img_size[0], img_size[0]), dtype=np.float16)
    indices = coords[:, 0], coords[:, 1], coords[:, 2]

    np.maximum.at(grid[0], indices, vals[:, 0])
    np.maximum.at(grid[1], indices, vals[:, 1])
    np.maximum.at(grid[2], indices, vals[:, 2])


    return grid

In [27]:
class PatientLevelTestset(Dataset):
    def __init__(self,
                 base_path: str,
                 dataframe: pd.DataFrame,
                 transform_3d=None):
        self.base_path = base_path

        self.dataframe = (dataframe[['study_id', "series_id", "series_description"]]
                          .drop_duplicates())

        self.subjects = self.dataframe[['study_id']].drop_duplicates().reset_index(drop=True)
        self.series_descs = {e[0]: e[1] for e in self.dataframe[["series_id", "series_description"]].drop_duplicates().values}

        self.transform_3d = transform_3d

    def __len__(self):
        return len(self.subjects)

    def __getitem__(self, index):
        curr = self.subjects.iloc[index]
        study_path = os.path.join(self.base_path, str(curr["study_id"]))

        study_images = read_study_as_voxel_grid(study_path, self.series_descs)

        if self.transform_3d is not None:
            study_images = self.transform_3d(torch.FloatTensor(study_images))
            return study_images.to(torch.half), str(curr["study_id"])

        return torch.HalfTensor(study_images), str(curr["study_id"])

In [28]:
def create_subject_level_fallback_testset_and_loader(df: pd.DataFrame,
                                             transform_3d,
                                             base_path: str,
                                             batch_size=1,
                                             num_workers=4):
    testset = PatientLevelTestset(base_path, df, transform_3d=transform_3d)
    test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    return testset, test_loader

In [29]:
test_descs_invalid = test_descs[~test_descs["study_id"].isin(patient_bounding_boxes["study_id"])]
testset_fallback, test_loader_fallback = create_subject_level_fallback_testset_and_loader(test_descs_invalid, transform_3d, test_images_basepath)

## Model Loading

In [30]:
class CNN_Model_3D_Multihead(nn.Module):
    def __init__(self,
                 backbone="efficientnet_lite0",
                 in_chans=1,
                 out_classes=5,
                 cutpoint_margin=0.15,
                 pretrained=False):
        super(CNN_Model_3D_Multihead, self).__init__()
        self.out_classes = out_classes

        self.encoder = timm_3d.create_model(
            backbone,
            features_only=False,
            drop_rate=0,
            drop_path_rate=0,
            pretrained=pretrained,
            in_chans=in_chans,
            global_pool="max"
        )
        if "efficientnet" in backbone:
            head_in_dim = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Sequential(
                nn.LayerNorm(head_in_dim),
                nn.Dropout(0),
            )

        elif "vit" in backbone:
            self.encoder.head.drop = nn.Dropout(0)
            head_in_dim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()

        self.heads = nn.ModuleList(
            [nn.Sequential(
                nn.Linear(head_in_dim, 1),
                LogisticCumulativeLink(3)
            ) for i in range(out_classes)]
        )

        self.ascension_callback = AscensionCallback(margin=cutpoint_margin)

    def forward(self, x):
        feat = self.encoder(x)
        return torch.swapaxes(torch.stack([head(feat) for head in self.heads]), 0, 1)

    def _ascension_callback(self):
        for head in self.heads:
            self.ascension_callback.clip(head[-1])

In [31]:
fallback_model = CNN_Model_3D_Multihead(backbone="maxvit_rmlp_tiny_rw_256", in_chans=3, out_classes=25).to(device)
fallback_model.load_state_dict(torch.load("/kaggle/input/rsna-2024/pytorch/vit_voxel_v2/7/maxvit_rmlp_tiny_rw_256_256_v2_fold_3_32.pt"))

/tmp/ipykernel_23/186658774.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fallback_model.load_state_dict(torch.load("/kaggle/input/rsna-2024/pytorch/vit_voxel_v2/7/max

<All keys matched successfully>

In [32]:
import cv2

start_time = time.time()

with torch.no_grad():
    with autocast(dtype=torch.float16):
        fallback_model.eval()

        for images, study_id in test_loader_fallback:
            output = fallback_model(images.to(device))
            for i, batch_out in enumerate(output):
                batch_out = output.cpu().numpy()[i]
                for index, level in enumerate(batch_out):
                    row_id = f"{study_id[i]}_{ALL_CONDITIONS[index // 5]}_{LEVELS[index % 5]}"
                    results_df.loc[results_df.row_id == row_id,'normal_mild'] = level[0]
                    results_df.loc[results_df.row_id == row_id,'moderate'] = level[1]
                    results_df.loc[results_df.row_id == row_id,'severe'] = level[2]
                
print("--- %s seconds ---" % (time.time() - start_time))

/tmp/ipykernel_23/421822995.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.float16):


--- 0.1591629981994629 seconds ---


In [33]:
results_df

,row_id,normal_mild,moderate,severe
0,44036939_left_neural_foraminal_narrowing_l1_l2,0.982586,0.017113,0.000301
1,44036939_left_neural_foraminal_narrowing_l2_l3,0.390407,0.583738,0.025856
2,44036939_left_neural_foraminal_narrowing_l3_l4,0.022515,0.552866,0.424619
3,44036939_left_neural_foraminal_narrowing_l4_l5,0.008811,0.334560,0.656629
4,44036939_left_neural_foraminal_narrowing_l5_s1,0.148875,0.762552,0.088573
5,44036939_left_subarticular_stenosis_l1_l2,0.412234,0.514634,0.073132
6,44036939_left_subarticular_stenosis_l2_l3,0.070129,0.506652,0.423220
7,44036939_left_subarticular_stenosis_l3_l4,0.031522,0.338817,0.629661
8,44036939_left_subarticular_stenosis_l4_l5,0.030380,0.331123,0.638497
9,44036939_left_subarticular_stenosis_l5_s1,0.011912,0.166972,0.821116


In [34]:
results_df.to_csv('submission.csv', index=False)